In [18]:
import numpy as np
import scipy
import random
import qiskit 
from qiskit import QuantumCircuit, transpile
from qiskit_aer import Aer, AerSimulator
from qiskit.circuit.library import UnitaryGate
from qiskit.visualization import array_to_latex
from mgbenchmark.main import mg_unitary_to_so, generate_jw_list, generate_jw_basis, unitary_to_superoperator, compound_matrix, mg_so_to_superoperator
from mgbenchmark.utils import generate_binary_strings, superop_to_dictionaries, dictionary_to_distribution, string_to_pauli, generate_jw_list_pauli, generate_jw_list

## Efficient Matchgate Tomography

Below is a demo for performing efficient matchgate tomography, as discussed in section VI of [our paper](https://arxiv.org/abs/2404.07974). It is a simple implementation of steps 3-5 of the algorithm to calculate matrix elements $R$, which then determine the matchgate unitary $U$.

It is equivalent to an implementation of the FLO (Fermionic Linear Optics) tomography algorithm of [Oszmaniec, Dangniam, Morales and Zimboras](https://arxiv.org/abs/2012.15825) who proved that $\mathcal{O}(n^3 \log n)$ repetitions suffice to correctly reconstruct the unitary $U$.




We start off by generating a random $R \in SO(2n)$ matrix using the Haar Measure. We then calculate its quadratic hamiltonian elements

$h_{ij} = \frac{1}{4} \log(R)_{ij}$

and from that, find the quadratic Hamiltonian:

$H = i \sum_{i \neq j} h_{i j} c_i c_j$.

($c_i$ are the Clifford Algebra generators in the Jordan-Wigner basis). 

We then find the corresponding unitary:

$U = e^{-iH}$

In [3]:
n=4
jwlist = generate_jw_list(n)
S = scipy.stats.special_ortho_group.rvs(2*n)
log_x = scipy.linalg.logm(S) / 4

quadh = np.zeros((2**n, 2**n))
for i in range(len(log_x)):
    for j in range(len(log_x)):
        if i != j:
            quadh = quadh + ( jwlist[i] @ jwlist[j] ) * log_x[i][j] * 1j

U = scipy.linalg.expm(-1j * quadh)


Below is code (adapted from the matchgate benchmarking demo) to estimate the matrix elements

$R_{ij} = \frac{1}{2^n} \text{Tr}(c_i^\dagger U c_j U^\dagger)$.

Each matrix element is estimated using $ \lceil n \ln (n / \delta) / \epsilon^2 \rceil$ shots, where $\epsilon$ is the precision and $\delta$ the failure probability.

This is carried out $4n^2$ times, giving a total number of shots of $4n^3 \lceil \ln (n / \delta) / \epsilon^2 \rceil $.

In [6]:
# Specify key parameters:
epsilon = 0.05
delta = 0.01


# Generate the superoperator and the dictionaries
ugate = UnitaryGate(U, label="U")

shots = 0
progcount = 0
jw_list = generate_jw_list_pauli(n)
str_list = generate_binary_strings(2*n, 1)


R = np.zeros((2*n, 2*n),dtype = 'complex128')

for x in range(2*n):
    for y in range(2*n):

        jobs_dict = {}
        circuit_dict = {}
        phase_dict = {}

        # Input iterated binary string 
        s_pair = str_list[x], str_list[y]

        # Process the input string into pauli + phase
        input_pauli = string_to_pauli(jw_list, s_pair[1])
        input_phase = input_pauli.phase
        input_pauli.phase = 0
        i_string = str(input_pauli)

        # Process the output string into pauli + phase
        output_pauli = string_to_pauli(jw_list, s_pair[0]).adjoint()
        output_phase = output_pauli.phase
        output_pauli.phase = 0
        j_string = str(output_pauli)

        phase = (-1j) ** (input_phase + output_phase)

        n_sample = int(np.ceil(n * np.log(n / delta) / ( epsilon ** 2)))

        tr = 0

        for _m in range(n_sample):

            # Create the quantum circuit
            qci = QuantumCircuit(n,n)
            lamb = 1
            qci_identifier = s_pair
            qci_key = ""

            # INITIALISE THE CIRCUIT IN PAULI EIGENSTATE
            # Apply the cliffords to the circuit according to the input. 
            # This prepares an eigenstate of the pauli & stores the eigenvalue as lamb
            # NOTE: The indices are reversed due to the Qiskit Convention, so q_(n-1) represents the first qubit and q_0 the last 
            for i in range(len(i_string)):
                prob = random.randint(0,1)
                if prob == 0:
                    qci_key += "0"
                else:
                    qci_key += "1"

                if i_string[i] == 'I':
                    if prob == 0:
                        qci.id(n-1-i)
                    else: 
                        qci.x(n-1-i)
                elif i_string[i] == 'Z':
                    if prob == 0:
                        qci.x(n-1-i)
                        lamb = lamb * -1
                    else:
                        qci.id(n-1-i)
                elif i_string[i] == 'X':
                    if prob == 0:
                        qci.h(n-1-i)
                    else:
                        qci.x(n-1-i)
                        qci.h(n-1-i)
                        lamb = lamb * -1
                elif i_string[i] == 'Y':
                    if prob == 0:
                        qci.h(n-1-i)
                        qci.s(n-1-i)
                    else:
                        qci.x(n-1-i)
                        qci.h(n-1-i)
                        qci.s(n-1-i)
                        lamb = lamb * -1

            # APPEND THE UNITARY GATE TO INPUT STATE
            qci.barrier()
            qci.append(ugate, list(range(n)))
            qci.barrier()

            # MEASURE THE OUTPUT IN PAULI BASIS
            # Apply the cliffords to the circuit according to the output string. This will give measurements in the pauli basis
            for j in range(len(j_string)):
                if j_string[j] == "I" or j_string[j] == "Z":
                    qci.id(n-1-j)
                elif j_string[j] == "X":
                    qci.h(n-1-j)
                elif j_string[j] == "Y":
                    qci.sdg(n-1-j)
                    qci.h(n-1-j)

            for j in range(len(j_string)):
                if j_string[j] == 'X' or j_string[j] == 'Y' or j_string[j] == 'Z':
                    qci.measure(n-1-j, n-1-j)

            qci_identifier += (qci_key,)

            if qci_identifier not in jobs_dict:
                jobs_dict[qci_identifier] = 1
                phase_dict[qci_identifier] = (lamb * phase)
                circuit_dict[qci_identifier] = qci
            elif qci_identifier in jobs_dict:
                jobs_dict[qci_identifier] += 1
            
            shots += 1

        # (The code has been optimised to 'tally' circuit simulations together, to reduce the number of calls to the backend.)
        for qci_identifier in jobs_dict:
            simulator = Aer.get_backend('aer_simulator')
            transpiled_circuit = transpile(circuit_dict[qci_identifier], backend=simulator)
            job = simulator.run(transpiled_circuit, shots=jobs_dict[qci_identifier])
            result = job.result()
            counts = result.get_counts()
            for strings in counts:
                tr += (-1)**strings.count('1') * phase_dict[qci_identifier] * counts[strings]

        progcount += 1
        
        print(progcount * 100 / (2*n)**2, "% complete")
        R[x,y] = tr / n_sample

print("shots:", shots)



1.5625 % complete
3.125 % complete
4.6875 % complete
6.25 % complete
7.8125 % complete
9.375 % complete
10.9375 % complete
12.5 % complete
14.0625 % complete
15.625 % complete
17.1875 % complete
18.75 % complete
20.3125 % complete
21.875 % complete
23.4375 % complete
25.0 % complete
26.5625 % complete
28.125 % complete
29.6875 % complete
31.25 % complete
32.8125 % complete
34.375 % complete
35.9375 % complete
37.5 % complete
39.0625 % complete
40.625 % complete
42.1875 % complete
43.75 % complete
45.3125 % complete
46.875 % complete
48.4375 % complete
50.0 % complete
51.5625 % complete
53.125 % complete
54.6875 % complete
56.25 % complete
57.8125 % complete
59.375 % complete
60.9375 % complete
62.5 % complete
64.0625 % complete
65.625 % complete
67.1875 % complete
68.75 % complete
70.3125 % complete
71.875 % complete
73.4375 % complete
75.0 % complete
76.5625 % complete
78.125 % complete
79.6875 % complete
81.25 % complete
82.8125 % complete
84.375 % complete
85.9375 % complete
87.5 % 

After getting an estimate $\tilde{R}$ of $R$, we repeat the previous calculations to find $\tilde{U}$, the estimated unitary:

In [7]:
jwlist = generate_jw_list(n)
log_x = scipy.linalg.logm(R) / 4

quadh = np.zeros((2**n, 2**n))
for i in range(len(log_x)):
    for j in range(len(log_x)):
        if i != j:
            quadh = quadh + ( jwlist[i] @ jwlist[j] ) * log_x[i][j] * 1j

U_calc = scipy.linalg.expm(-1j * quadh)

We now find the determinant of $R$, the $L2$ norm of $U - \tilde{U}$ and $R - \tilde{R}$, as well as the determinant norm $\sqrt{\det | \tilde{U} - U |}$.

In [9]:
np.linalg.det(R)

(1.0344101341531569+0j)

In [10]:
np.linalg.norm(U-U_calc, ord=2)

0.02784979556947163

In [11]:
np.linalg.norm(R-S, ord=2)

0.047644325174019515

In [12]:
np.sqrt(np.linalg.det(U-U_calc))

(7.844610176163786e-18-6.749246335167192e-30j)

The true and estimated matrices $R$, $U$ are printed below:

In [13]:
array_to_latex(S)

<IPython.core.display.Latex object>

In [14]:
array_to_latex(R)

<IPython.core.display.Latex object>

In [15]:
array_to_latex(S- R)

<IPython.core.display.Latex object>

In [16]:
array_to_latex(U, max_size=100)

<IPython.core.display.Latex object>

In [17]:
array_to_latex(U - U_calc)

<IPython.core.display.Latex object>